In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from tqdm import tqdm
import pickle

class_names = ['Normal', 'Epidural', 'Intraparenchymal', 'Intraventricular', 'Subarachnoid', 'Subdural']

In [ ]:
rsna_series = pd.read_csv('../input/rsna-abnormal-series-labels/RSNA_Series.csv')
rsna_series = rsna_series.loc[rsna_series['Normal']==0]
rsna_series.head()

In [ ]:
rsna_df = pd.read_csv('../input/rsna-cq500-data-frames-png/RSNA.csv')
rsna_df.head()

In [ ]:
root_path = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'

In [ ]:
rsna_ids = rsna_series['UID'].values
rsna_dict = {}
for i in tqdm(range(len(rsna_ids))):
    image_path = rsna_series.loc[rsna_series['UID']==rsna_ids[i]]['ImageList'].values[0]
    image_path = image_path[1:-1].split(',')
    image_path = [x.replace(' \'', '').replace('\'', '') for x in image_path]
    image_df = rsna_df.loc[rsna_df['imgfile'].isin(image_path)]
    image_labels = []
    dicom_path = []
    for j in range(len(image_path)):
        image_labels.append(list(image_df.loc[image_df['imgfile']==image_path[j]][class_names].values[0]))
        dicom_path.append(root_path+image_path[j].split('/')[-1].split('.')[0]+'.dcm')
    rsna_dict[rsna_ids[i]] = [dicom_path, image_labels]

In [ ]:
with open('RSNA_Abnormal_Series_DICOM.pkl', 'wb') as f:
    pickle.dump(rsna_dict, f)

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf
import shutil
from skimage import morphology
from scipy import ndimage


def window_img(img, img_min, img_max):
    img = np.clip(img, img_min, img_max)
    return img

def window_3d(image, window=None):
    if window:
        image_total = np.empty((len(image), 256, 256))
        for i in range(len(image)):
            image_brain =  window_img(image[i], 0, 80)
            image_brain = (image_brain - 0) / 80
            image_total[i] = cv2.resize(image_brain, dsize=(256, 256))
    else:
        image_total = np.empty((len(image), 256, 256, 3))
        for i in range(len(image)):
            image_brain =  window_img(image[i], 0, 80)
            image_brain = (image_brain - 0) / 80
            image_brain = np.expand_dims(image_brain, axis=-1)

            image_subdural =  window_img(image[i], -20, 180)
            image_subdural = (image_subdural - (-20)) / 200
            image_subdural = np.expand_dims(image_subdural, axis=-1)

            image_soft =  window_img(image[i], 20, 60)
            image_soft = (image_soft - (20)) / 40
            image_soft = np.expand_dims(image_soft, axis=-1)

            image_total[i] = cv2.resize(np.concatenate([image_brain, image_subdural, image_soft], axis=-1), dsize=(256, 256))
    return (image_total*255).astype(np.uint8)


def remove_noise(img):
    image = []
    for brain_image in img:
        segmentation = morphology.dilation(brain_image, np.ones((1, 1)))
        labels, label_nb = ndimage.label(segmentation)

        label_count = np.bincount(labels.ravel().astype(np.uint8))
        label_count[0] = 0

        mask = labels == label_count.argmax()

        # Improve the brain mask
        mask = morphology.dilation(mask, np.ones((1, 1)))
        mask = ndimage.morphology.binary_fill_holes(mask)
        mask = morphology.dilation(mask, np.ones((3, 3)))
        image.append(mask * brain_image)

    return np.array(image)

def skull_strip(img_clipp, img_orig):
    img_clipp[img_clipp == np.amax(img_clipp)] = 0
    image = []
    for hi in img_clipp:
        labels, label_nb = ndimage.label(hi)
        label_count = np.bincount(labels.ravel().astype(np.uint8))
        label_count[0] = 0
        mask = labels == label_count.argmax()
        mask = morphology.dilation(mask, np.ones((1, 1)))
        mask = ndimage.morphology.binary_fill_holes(mask)
        mask = morphology.dilation(mask, np.ones((3, 3)))
        mask = ndimage.morphology.binary_fill_holes(mask)
        image.append(mask * hi)
    image = np.array(image)
    mask = (image > 0).astype(np.uint8)
    mask = ndimage.binary_fill_holes(mask)
    img_orig[mask==0] = 0
    return img_orig

def crop_resize_image(image, label):
    out_image = []
    out_label = []
    for i, img in enumerate(image):
        mask = img == 0
        coords = np.array(np.nonzero(~mask))
        if coords.shape[1] > 100:
            top_left = np.min(coords, axis=1)
            bottom_right = np.max(coords, axis=1)
            croped_image = img[top_left[0]:bottom_right[0],
                            top_left[1]:bottom_right[1]]
            if 0 in croped_image.shape:
                continue
            croped_image = np.expand_dims(croped_image, axis=-1)
            resized= np.array(tf.image.resize_with_pad(croped_image,
                                                    256, 256, method='bilinear',antialias=False))
            resized = np.squeeze(resized)
            out_image.append(resized)
            out_label.append(label[i])
    return np.array(out_image), out_label
    
def _read_dicom(dicoms, labels):
    image = []
    for dicom in dicoms:
        img = sitk.ReadImage(dicom)
        img = np.array(sitk.GetArrayFromImage(img))[0]
        image.append(img)
    image = np.array(image)
    img_clip = np.clip(image, 0, 80)
    noise_removed = remove_noise(img_clip)
    img_stripped = skull_strip(noise_removed, image)
    img_resized, label = crop_resize_image(img_stripped, labels)
    img_final = window_3d(img_resized)
    return img_final, label

In [ ]:
import zipfile

print('creating archive')
zf = zipfile.ZipFile('RSNA_Abnormal_Images.zip', mode='w')

In [ ]:
idds = list(rsna_dict.keys())
rsna_png_dict = {}
for i in tqdm(range(len(idds))):
    dicoms = rsna_dict[idds[i]][0]
    labels = rsna_dict[idds[i]][1]
    image_array, labels = _read_dicom(dicoms, labels)
    png_series_name = []
    for j in range(len(image_array)):
        img_name = dicoms[j].split('/')[-1].split('.')[0]+'.png'
        img_path = '{}/{}'.format(idds[i], img_name)
        png_series_name.append(img_path)
        cv2.imwrite(img_name, image_array[j])
        zf.write(img_name, img_path, zipfile.ZIP_DEFLATED)
        os.remove(img_name)
    rsna_png_dict[idds[i]] = [png_series_name, labels]

In [ ]:
for i in range(len(image_array)):
    plt.figure()
    plt.imshow(image_array[i], cmap='bone')
    plt.title(class_names[np.argmax(labels[i])])
    plt.show()

In [ ]:
zf.close()

In [ ]:
with open('RSNA_Abnormal_Series_PNG.pkl', 'wb') as f:
    pickle.dump(rsna_png_dict, f)